In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46085
2,Huelva,Confirmados PDIA 14 días,340
3,Huelva,Tasa PDIA 14 días,"66,25484732155036"
4,Huelva,Confirmados PDIA 7 días,173
5,Huelva,Tasa PDIA 7 dias,"33,71202525478886"
6,Huelva,Total Confirmados,46288
7,Huelva,Curados,45071
8,Huelva,Fallecidos,419


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46085.0


/tmp/ipykernel_23370/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12663.0


/tmp/ipykernel_23370/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_23370/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_23370/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_23370/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4640 personas en los últimos 7 días 

Un positivo PCR cada 2213 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46085.0,173.0,340.0,513170.0,33.712025,66.254847,48.0
Huelva-Costa,27261.0,100.0,199.0,289548.0,34.536588,68.727810,30.0
Condado-Campiña,14311.0,50.0,89.0,156231.0,32.003892,56.966927,11.0
Cartaya,2111.0,31.0,68.0,20083.0,154.359408,338.594831,10.0
Huelva (capital),12663.0,31.0,65.0,143837.0,21.552174,45.190042,10.0
Sierra de Huelva-Andévalo Central,4087.0,23.0,52.0,67391.0,34.129186,77.161639,7.0
Campofrío,50.0,15.0,28.0,713.0,2103.786816,3927.068724,5.0
Almonte,2323.0,13.0,16.0,24507.0,53.046068,65.287469,4.0
Moguer,2013.0,11.0,26.0,21867.0,50.304111,118.900627,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,50.0,15.0,28.0,713.0,2103.786816,3927.068724,5.0
Granado (El),46.0,5.0,6.0,532.0,939.849624,1127.819549,1.0
Cartaya,2111.0,31.0,68.0,20083.0,154.359408,338.594831,10.0
Zalamea la Real,123.0,1.0,7.0,3054.0,32.743942,229.207597,0.0
Minas de Riotinto,184.0,3.0,6.0,3812.0,78.698846,157.397692,0.0
Palos de la Frontera,1065.0,10.0,17.0,11742.0,85.164367,144.779424,0.0
Cabezas Rubias,53.0,1.0,1.0,706.0,141.643059,141.643059,NaN
Moguer,2013.0,11.0,26.0,21867.0,50.304111,118.900627,4.0
San Bartolomé de la Torre,382.0,4.0,4.0,3761.0,106.354693,106.354693,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Zalamea la Real,123.0,1.0,7.0,3054.0,32.743942,229.207597,0.0,0.142857
Bollullos Par del Condado,1297.0,1.0,5.0,14387.0,6.950719,34.753597,0.0,0.200000
Punta Umbría,1277.0,1.0,3.0,15355.0,6.512537,19.537610,0.0,0.333333
Nerva,254.0,1.0,3.0,5169.0,19.346102,58.038305,1.0,0.333333
Palma del Condado (La),1434.0,4.0,10.0,10801.0,37.033608,92.584020,0.0,0.400000
Moguer,2013.0,11.0,26.0,21867.0,50.304111,118.900627,4.0,0.423077
Ayamonte,2106.0,7.0,16.0,21104.0,33.169067,75.815011,1.0,0.437500
Sierra de Huelva-Andévalo Central,4087.0,23.0,52.0,67391.0,34.129186,77.161639,7.0,0.442308
Cartaya,2111.0,31.0,68.0,20083.0,154.359408,338.594831,10.0,0.455882
